# Messaging


### Imports

In [1]:
from aries_cloudcontroller import AriesAgentController
from libs.messaging_service import MessagingService
import os
from termcolor import colored

### Initialise the Agent Controller

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://hlgf-agent:3021 and an api key of adminApiKey


### Start a Webhook Server

In [3]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


## Instatiate Messaging Service

In [4]:
messaging_service = MessagingService(agent_controller)

### If you saved the state of the service previously you can load it from file

In [5]:
messaging_service.load_from_file()

--------------------------------------------------
1 Contacts
--------------------------------------------------
Contact Demo Participant
ID: f0b85c02-4560-42d3-8439-92efb6c53424 
Label: None
Status : ACTIVE 
Messages 3 
--------------------------------------------------



In [8]:
## Identify the conctact you want to interact with
loaded_contact_id = "f0b85c02-4560-42d3-8439-92efb6c53424"

In [11]:
messaging_service.display_inbox_for_contact(loaded_contact_id)

----------------------------------------------------------------------------------------------------
Inbox for Contact Demo Participant
----------------------------------------------------------------------------------------------------
Connection ID : f0b85c02-4560-42d3-8439-92efb6c53424 
Agent Label : None 
Total Messages : 4
----------------------------------------------------------------------------------------------------
Received                                                                              Sent
----------------------------------------------------------------------------------------------------
Tue Jun  8 07:55:46 2021
Hey how are you, want to grab a coffee? 
----------------------------------------------------------------------------------------------------
Tue Jun  8 07:55:50 2021
Hey how are you, want to grab a coffee? 
----------------------------------------------------------------------------------------------------
Tue Jun  8 07:55:53 2021
                  

In [10]:
message = "Hey, I just reloaded my state. Hope I can still send you messages!"
messaging_service.send_message(loaded_contact_id, message)

## Optional - Configure Mediator

In [ ]:
mediator_invite = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'bb47c3ea-ab9a-4ef4-984d-409cb4909956', 'recipientKeys': ['HkwRkbCGEXn4LviXNen5f8kqQ1WxVRpQQGxaQG7PuSZD'], 'serviceEndpoint': 'http://139.162.224.50:6666', 'label': 'HGF_ACA_Playground_Mediator'}

In [ ]:
invitation_response = await agent_controller.connections.receive_invitation(mediator_invite)

In [ ]:
connection_id = invitation_response["connection_id"]
### check state of connection
connection = await agent_controller.connections.get_connection(connection_id)

# Let's check for the state
if connection['state'] != 'active':
    print("No active connection. \n Please go back and ensure you have established an active connection")    
else:
    ## request mediation
    mediation_req = await agent_controller.mediation.request_mediation(connection_id)
    

In [ ]:
### To have a look at the mediation records and we should see our mediation in there
records_response = await agent_controller.mediation.get_mediation_records()
print(f"State : {records_response[0]['state']}")

In [ ]:
default_mediation_res = await agent_controller.mediation.set_default_mediator(records_response[0]['mediation_id'])
print(default_mediation_res)

## Add New Contacts

### Either Create an Invitation and Share

In [ ]:
alias = "Demo Participant"
response = messaging_service.new_contact_invite(alias)

# If you want to display as Q.R for example
invitation = response["invitation"]

new_contact_id = response["connection_id"]

### Or Add Contact using Invitation You have been sent

In [ ]:
invitation = {"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "79690dce-73e8-40fe-943d-212771bfce94", "recipientKeys": ["GjM12SGxGiyN4TJ3XLda8AjNUPSxsS8byDnfN4Uvpv5H"], "serviceEndpoint": "http://139.162.224.50:6666", "routingKeys": ["BQKTbHwJap7Su9ZtkBJgqnHEAQt4ysjygBkAEjEngMfp"], "label": "<Label YOUR Agent>"}

In [ ]:
new_contact_id = messaging_service.accept_contact_invitation(invitation, "Demo Participant")

## Send Message

In [ ]:
some_arbitrary_sized_message = "Hello"
messaging_service.send_message(new_contact_id, some_arbitrary_sized_message)

In [6]:
messaging_service.display_inbox_for_contact(new_contact_id)

NameError: name 'new_contact_id' is not defined

In [ ]:
messaging_service.save_to_file()

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()